In [4]:
import qualifire
import os
from dotenv import load_dotenv

load_dotenv()

QUALIFIRE_API_KEY = os.getenv("QUALIFIRE_API_KEY")

if not QUALIFIRE_API_KEY or QUALIFIRE_API_KEY == "YOUR QUALIFIRE API KEY":
    print("Please replace 'YOUR QUALIFIRE API KEY' with your actual key.")
    raise ValueError("QUALIFIRE_API_KEY is not set")
else:
    qualifire.init(
        api_key=QUALIFIRE_API_KEY,
    )
    print("Qualifire SDK Initialized Successfully!")

Qualifire SDK Initialized Successfully!


In [5]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY or OPENAI_API_KEY == "YOUR OPENAI API KEY":
    print("Please replace 'YOUR OPENAI API KEY' with your actual key for the agent to run.")
    raise ValueError("OPENAI_API_KEY is not set")
else:
    print("OpenAI API Key set.")
# Note: The agent code itself will pass this to the init_chat_model function.
# Setting it as an environment variable here is good practice if other parts of LangGraph expect it.
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

OpenAI API Key set.


In [ ]:
import requests

from LangGraph.chat_models import init_chat_model
from LangGraph_community.utilities import SQLDatabase
from LangGraph_community.agent_toolkits import SQLDatabaseToolkit
from langgraph.prebuilt import create_react_agent

print("Imported necessary libraries for the agent.")

# Check if API keys are set before proceeding
if QUALIFIRE_API_KEY == "YOUR QUALIFIRE API KEY" or OPENAI_API_KEY == "YOUR OPENAI API KEY":
    print("\nERROR: API keys are not set. Please set QUALIFIRE_API_KEY and OPENAI_API_KEY in the cells above.\n")
else:
    print("\nAPI keys seem to be set. Proceeding with agent initialization.\n")
 
    # Initialize the LLM
    llm = init_chat_model(
        "openai:gpt-4.1", # This can be ANY model supported by your init_chat_model
        api_key=OPENAI_API_KEY,
        base_url="https://proxy.qualifire.ai/api/providers/openai/", # Needed for moderation we'll discuss that in the next tutorial
        default_headers={
            "X-Qualifire-API-Key": QUALIFIRE_API_KEY
        },
    )
    print("LLM Initialized.")

    # Download the Chinook database
    db_file_path = "./assets/Chinook.db"

    # Setup SQL Database
    db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")

    # Setup SQL Database Toolkit
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    tools = toolkit.get_tools()
    print("\nAvailable Tools:")
    for tool in tools:
        print(f"- {tool.name}: {tool.description}")

    # Define the system prompt for the agent
    system_prompt_string = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
        dialect=db.dialect,
        top_k=5,
    )

    # Create the ReAct Agent
    agent_executor = create_react_agent(
        llm,
        tools,
        prompt=system_prompt_string,
    )

    print("\nAgent executor created.")

    # Define a question and stream the agent's response
    question = "Which sales agent made the most in sales in 2009?"
    print(f"\nQuestion: {question}\n")
    print("Streaming agent response:")
    print("-" * 30)

    try:
        for step in agent_executor.stream(
            {"messages": [{"role": "user", "content": question}]},
            stream_mode="values", # "values" provides the full state at each step
        ):
            # The 'messages' key contains a list of AIMessage, HumanMessage, etc.
            # We're interested in the last message, which is usually the agent's thought or response.
            if "messages" in step and step["messages"]:
                step["messages"][-1].pretty_print()
                print("-" * 30)
    except Exception as e:
        print(f"An error occurred during agent execution: {e}")
        print("This might be due to API key issues, network problems, or unexpected agent behavior.")
        print("Please check your API keys and network connection.")
        print("If the agent itself has an issue, the Qualifire traces (if captured) might provide insights.")
